In [1]:
import os
USER = !whoami
USER = USER[0]
OUTPUT_PATH_BASE = '/user/{USER}'.format(USER=USER)

In [2]:
%%writefile mrjob_systems_test.py
#!/opt/anaconda/bin/python
from mrjob.compat import jobconf_from_env
from mrjob.job import MRJob


class MRCountLinesByFile(MRJob):

    def mapper(self, _, line):
        chars = len(line)
        words = len(line.split())
        lines = 1
        self.increment_counter('Item Counters', 'Lines', lines)
        self.increment_counter('Item Counters', 'Words', words)
        self.increment_counter('Item Counters', 'Characters', chars)
        yield "chars", chars
        yield "words", words
        yield "lines", lines

    def reducer(self, key, values):
        yield key, sum(values)


if __name__ == '__main__':
    MRCountLinesByFile.run()

Overwriting mrjob_systems_test.py


In [3]:
%%writefile stuff.txt
Here I am in the altiscale
Reading the jupyter notebook

Overwriting stuff.txt


In [4]:
!python mrjob_systems_test.py stuff.txt 

Using configs in /home/winegarj/.mrjob.conf
Creating temp directory /tmp/mrjob_systems_test.winegarj.20171003.233225.513513
Running step 1 of 1...
Counters: 3
	Item Counters
		Characters=54
		Lines=2
		Words=10
Counters: 3
	Item Counters
		Characters=54
		Lines=2
		Words=10
Streaming final output from /tmp/mrjob_systems_test.winegarj.20171003.233225.513513/output...
"chars"	54
"lines"	2
"words"	10
Removing temp directory /tmp/mrjob_systems_test.winegarj.20171003.233225.513513...


In [5]:
OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}
!python mrjob_systems_test.py \
        -r hadoop stuff.txt \
        --output-dir={OUTPUT_PATH} \
        --no-output

17/10/03 23:32:27 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 5760 minutes, Emptier interval = 360 minutes.
Moved: 'hdfs://nn-ia.s3s.altiscale.com:8020/user/winegarj/tests' to trash at: hdfs://nn-ia.s3s.altiscale.com:8020/user/winegarj/.Trash/Current
Using configs in /home/winegarj/.mrjob.conf
Using Hadoop version 2.7.3
Creating temp directory /tmp/mrjob_systems_test.winegarj.20171003.233227.950756
Copying local files to hdfs:///user/winegarj/tmp/mrjob/mrjob_systems_test.winegarj.20171003.233227.950756/files/...
Running step 1 of 1...
  packageJobJar: [] [/opt/hadoop-2.7.3/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar] /tmp/streamjob7597047788158342884.jar tmpDir=null
  Timeline service address: http://rm-ia.s3s.altiscale.com:8188/ws/v1/timeline/
  Connecting to ResourceManager at rm-ia.s3s.altiscale.com/10.251.255.108:8032
  Connecting to Application History server at rm-ia.s3s.altiscale.com/10.251.255.108:10200
  Timeline service address: http://

In [6]:
!hadoop fs -cat {OUTPUT_PATH}/*

"chars"	54
"lines"	2
"words"	10
